In [1]:
!pip install praw
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.1 MB/s eta 0:00:00


In [2]:
import praw
import requests
import os
import csv
# NEW: Import the Google Drive library
from google.colab import drive

# --- NEW: MOUNT YOUR GOOGLE DRIVE ---
# This will prompt you for authorization when you run it.
print("Connecting to Google Drive...")
drive.mount('/content/drive')
print("Google Drive connected successfully!")


# --- CONFIGURATION ---
# Step 1: Your Reddit API credentials
CLIENT_ID = "GqCbGMNlJsb-uNWMoYJxxA"
CLIENT_SECRET = "nzTshKEW6zUj890WtdqXIHb3gu9psg"
USER_AGENT = "VibeProjectMemeScraper/0.3 by fat_candy"

# Step 2: Define your target subreddits
TARGET_SUBREDDITS = [
    "IndiaSpeaks",
    "unitedstatesofindia",
    "librandu",
    "IndianDankMemes",
    "IndianMeyMeys",
    "Indiadiscussion",
    "okbhaibudbak",
    "2bharat4you",
    "bakchodi",
    "Sham_Sharma_Show",
    "IndianModerate",
    "IndiaDivine"
]

# Step 3: Customize your scrape
SORT_METHOD = 'top'
TIME_FILTER = 'year'
POST_LIMIT = 100

# --- UPDATED: Define the paths to save files in your Google Drive ---
# The main folder will be created in your Google Drive's root directory.
DRIVE_FOLDER_PATH = "/content/drive/My Drive/reddit_political_memes_pro"
DOWNLOAD_FOLDER = DRIVE_FOLDER_PATH
METADATA_FILE = os.path.join(DOWNLOAD_FOLDER, "metadata.csv")


# --- THE SCRIPT (No changes needed below this line) ---

def setup_praw():
    """Initializes and returns a PRAW Reddit instance."""
    print("Connecting to Reddit...")
    return praw.Reddit(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, user_agent=USER_AGENT)

def setup_environment():
    """Creates download folder and sets up the CSV file with headers."""
    if not os.path.exists(DOWNLOAD_FOLDER):
        print(f"Creating Google Drive folder: '{DOWNLOAD_FOLDER}'")
        os.makedirs(DOWNLOAD_FOLDER)

    with open(METADATA_FILE, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['subreddit', 'post_id', 'title', 'score', 'post_url', 'local_filename'])

def download_image(url, filepath):
    """Downloads an image from a URL and saves it."""
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filepath, 'wb') as f:
                f.write(response.content)
            return True
    except requests.exceptions.RequestException as e:
        print(f"   !! Error downloading {url}: {e}")
    return False

def log_metadata(subreddit, post, local_filename):
    """Appends a row to the metadata CSV file."""
    # We strip the main drive path to keep the CSV cleaner
    relative_filename = local_filename.replace(DOWNLOAD_FOLDER + '/', '')
    with open(METADATA_FILE, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([subreddit, post.id, post.title, post.score, f"https://www.reddit.com{post.permalink}", relative_filename])

def scrape_subreddits(reddit):
    """Main function to loop through subreddits and download content."""
    print(f"\n--- Starting Scrape (Mode: {SORT_METHOD}, Limit: {POST_LIMIT}) ---")
    total_downloads = 0

    for sub_name in TARGET_SUBREDDITS:
        sub_folder = os.path.join(DOWNLOAD_FOLDER, sub_name)
        if not os.path.exists(sub_folder):
            os.makedirs(sub_folder)

        print(f"\n[+] Scraping subreddit: r/{sub_name}")
        try:
            subreddit = reddit.subreddit(sub_name)

            if SORT_METHOD == 'top':
                posts = subreddit.top(time_filter=TIME_FILTER, limit=POST_LIMIT)
            elif SORT_METHOD == 'hot':
                posts = subreddit.hot(limit=POST_LIMIT)
            elif SORT_METHOD == 'new':
                posts = subreddit.new(limit=POST_LIMIT)
            else:
                print(f"   !! Invalid SORT_METHOD: {SORT_METHOD}. Skipping subreddit.")
                continue

            sub_download_count = 0
            for post in posts:
                if hasattr(post, 'is_gallery') and post.is_gallery:
                    gallery_count = 0
                    for i, item in enumerate(post.media_metadata.items()):
                        image_url = item[1]['s']['u']
                        file_ext = image_url.split('?')[0].split('.')[-1]
                        filename = f"{sub_name}_{post.id}_{i+1}.{file_ext}"
                        filepath = os.path.join(sub_folder, filename)

                        if not os.path.exists(filepath):
                            if download_image(image_url, filepath):
                                gallery_count += 1
                                log_metadata(sub_name, post, filepath)
                    if gallery_count > 0:
                        print(f"   -> Downloaded {gallery_count} images from gallery: {post.title[:50]}...")
                        sub_download_count += gallery_count

                elif post.url.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                    file_extension = os.path.splitext(post.url)[1]
                    filename = f"{sub_name}_{post.id}{file_extension}"
                    filepath = os.path.join(sub_folder, filename)

                    if not os.path.exists(filepath):
                        if download_image(post.url, filepath):
                            print(f"   -> Downloaded image: {filename}")
                            sub_download_count += 1
                            log_metadata(sub_name, post, filepath)

            print(f"[+] Downloaded {sub_download_count} new images from r/{sub_name}.")
            total_downloads += sub_download_count

        except Exception as e:
            print(f"   !! An error occurred with r/{sub_name}: {e}")

    print("\n--- Scrape Finished ---")
    print(f"Total new images downloaded: {total_downloads}")
    print(f"All files saved in your Google Drive at: '{DRIVE_FOLDER_PATH}'")


# --- Main Execution ---
if __name__ == "__main__":
    reddit_instance = setup_praw()
    setup_environment()
    scrape_subreddits(reddit_instance)

Connecting to Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive connected successfully!
Connecting to Reddit...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Creating Google Drive folder: '/content/drive/My Drive/reddit_political_memes_pro'

--- Starting Scrape (Mode: top, Limit: 100) ---

[+] Scraping subreddit: r/IndiaSpeaks


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

   -> Downloaded image: IndiaSpeaks_1fbeef3.jpeg


...

   -> Downloaded image: IndiaDivine_1f8gai8.jpeg


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



   -> Downloaded image: IndiaDivine_1fcrn74.png


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



   -> Downloaded image: IndiaDivine_1f86yun.png
[+] Downloaded 9 new images from r/IndiaDivine.

--- Scrape Finished ---
Total new images downloaded: 393
All files saved in your Google Drive at: '/content/drive/My Drive/reddit_political_memes_pro'
